# Image classification using TensorFlow

### Springboard Career Track Capstone Project 2

#### Linear Model in TensorFlow

#### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline 
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.pylabtools import figsize
figsize(12, 8)

#### Import data

Data acquired from https://www.cs.toronto.edu/~kriz/cifar.html

In [2]:
import tarfile
tar = tarfile.open("cifar-10-python.tar.gz")
tar.extractall()
tar.close()

#### Create function to unpickle data

In [3]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [4]:
def one_hot_encode(x):
    """
    One hot encode a list of sample labels. Return a one-hot encoded vector for each label.
    : x: List of sample Labels
    : return: Numpy array of one-hot encoded labels
    """  
    from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
    encoder = LabelBinarizer()
    encoder.fit(range(10))

    return encoder.transform(x)

#### Assign batch names, unpickle batches, and concatenate

In [5]:
f1 = 'cifar-10-batches-py\data_batch_1'
f2 = 'cifar-10-batches-py\data_batch_2'
f3 = 'cifar-10-batches-py\data_batch_3'
f4 = 'cifar-10-batches-py\data_batch_4'
f5 = 'cifar-10-batches-py\data_batch_5'
x_test = 'cifar-10-batches-py\\test_batch'
label_names = 'cifar-10-batches-py\\batches.meta'

In [6]:
b1 = unpickle(f1)

In [7]:
b1.keys()

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])

In [8]:
train_y1 = b1.get(b'labels')
b1data = b1.get(b'data')

b2 = unpickle(f2)
train_y2 = b2.get(b'labels')
b2data = b2.get(b'data')

b3 = unpickle(f3)
train_y3 = b3.get(b'labels')
b3data = b3.get(b'data')

b4 = unpickle(f4)
train_y4 = b4.get(b'labels')
b4data = b4.get(b'data')

b5 = unpickle(f5)
train_y5 = b5.get(b'labels')
b5data = b5.get(b'data')

x_train = np.concatenate((b1data, b2data, b3data, b4data, b5data), axis=0)
y_train = np.concatenate((train_y1, train_y2, train_y3, train_y4, train_y5), axis=0)

y_train_cls = y_train
y_train = one_hot_encode(y_train)

test = unpickle(x_test)
x_test = test.get(b'data')
y_test_cls = test.get(b'labels')
y_test = one_hot_encode(y_test_cls)
label_names = unpickle(label_names)

In [82]:
x_train.shape

(50000, 3072)

In [29]:
x_train

array([[ 59,  43,  50, ..., 140,  84,  72],
       [154, 126, 105, ..., 139, 142, 144],
       [255, 253, 253, ...,  83,  83,  84],
       ..., 
       [ 35,  40,  42, ...,  77,  66,  50],
       [189, 186, 185, ..., 169, 171, 171],
       [229, 236, 234, ..., 173, 162, 161]], dtype=uint8)

In [30]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ..., 
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [31]:
y_train.shape

(50000, 10)

In [32]:
label_names

{b'label_names': [b'airplane',
  b'automobile',
  b'bird',
  b'cat',
  b'deer',
  b'dog',
  b'frog',
  b'horse',
  b'ship',
  b'truck'],
 b'num_cases_per_batch': 10000,
 b'num_vis': 3072}

In [33]:
x_test.shape

(10000, 3072)

In [34]:
y_test.shape

(10000, 10)

 #### Example from https://github.com/martin-gorner/tensorflow-mnist-tutorial/blob/master/mnist_1.0_softmax.py
 https://docs.google.com/presentation/d/1TVixw6ItiZ8igjp6U17tcgoFrLSaHWQmMOwjlgQY9co/pub?slide=id.g110257a6da_0_598

Attempt 1 - Not convolutional

In [ ]:
learning_rate = .001
training_iteration = 30
batch_size = 100
display_step = 10

Initialize placeholder variables

In [ ]:
x = tf.placeholder(tf.float32, [None, 3072])
W = tf.Variable(tf.zeros([3072,10]))
b = tf.Variable(tf.zeros([10]))

y_true = tf.placeholder(tf.float32, [None, 10]) #placeholder from the true values
y_true_cls = tf.placeholder(tf.int64, [None])

init = tf.global_variables_initializer()

In [ ]:
logits = tf.matmul(x, W) + b 
y_pred = tf.nn.softmax(logits)
y_pred_cls = tf.argmax(y_pred,dimension=1)

# loss function
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y_true)
cost = tf.reduce_mean(cross_entropy)

# Add summary ops to collect data
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# % of correct answers found in batch
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
sess = tf.Session()
sess.run(init)

In [ ]:
for i in range(10):
    # load batch of images and correct answers
    batch_X, batch_Y,batch_Y_cls = x_train,y_train,y_train_cls
    feed_dict_train = {x: batch_X, y_true: batch_Y}
    feed_dict_train1 = {x: batch_X, y_true: batch_Y,y_true_cls: batch_Y_cls}
    
    # train
    sess.run(optimizer, feed_dict=feed_dict_train)
    a,c = sess.run([accuracy, cross_entropy], feed_dict=feed_dict_train1)
    a
    
feed_dict_test = {x: x_test,
                  y_true: y_test,
                  y_true_cls: y_test_cls}

from sklearn.metrics import confusion_matrix
sess.run(y_pred_cls, feed_dict=feed_dict_test)    
pred = sess.run(y_pred_cls, feed_dict=feed_dict_test)    
cm = confusion_matrix(y_true=y_test_cls,y_pred=pred)

In [ ]:
cm

Using Keras (adapted from https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html)

In [ ]:
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

In [ ]:
img = tf.placeholder(tf.float32, shape=(None, 3072))

In [ ]:
from keras.layers import Dense

# Keras layers can be called on TensorFlow tensors:
x = Dense(128, activation='relu')(img)  # fully-connected layer with 128 units and ReLU activation
x = Dense(128, activation='relu')(x)
preds = Dense(10, activation='softmax')(x)  # output layer with 10 units and a softmax activation

In [ ]:
labels = tf.placeholder(tf.float32, shape=(None, 10))

from keras.objectives import categorical_crossentropy
loss = tf.reduce_mean(categorical_crossentropy(labels, preds))

In [ ]:
# from tensorflow.examples.tutorials.mnist import input_data
# mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# Initialize all variables
init_op = tf.global_variables_initializer()
sess.run(init_op)

# Run training loop
with sess.as_default():
    for i in range(100):
        batch = x_train.next_batch(50)
        train_step.run(feed_dict={img: x_train,
                                  labels: y_train})

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
batch_size = 500

# Initialize all variables
init_op = tf.global_variables_initializer()
sess.run(init_op)

# Run training loop
with sess.as_default():
    for i in range(100):
        j = 0
        x_train_batch = x_train[j:j+batch_size,]
        y_train_batch = y_train[j:j+batch_size,]
        j = j + batch_size
        if j > x_train.shape[0]:
            j = 0
            
        train_step.run(feed_dict={img: x_train_batch,
                                  labels: y_train_batch})

In [ ]:
from keras.metrics import categorical_accuracy as accuracy

acc_value = accuracy(labels, preds)
with sess.as_default():
    print(acc_value.eval(feed_dict={img: x_test,
                                    labels: y_test}))

Using baseline model Keras example from https://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-neural-networks-python-keras/

In [23]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import keras.utils

In [10]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [11]:
num_pixels = 32 * 32 * 3
num_classes = 10

# build the model
model = baseline_model()
# Fit the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
83s - loss: 14.5071 - acc: 0.0999 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 2/10
83s - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 3/10
83s - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 4/10
83s - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 5/10
83s - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 6/10
82s - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 7/10
82s - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 8/10
83s - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 9/10
89s - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 10/10
87s - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000


Baseline Error: 90.00%


Run with normalized inputs

In [14]:
# normalize inputs from 0-255 to 0-1
x_train = x_train / 255
x_test = x_test / 255

In [15]:
num_pixels = 32 * 32 * 3
num_classes = 10

from keras import optimizers

# build the model
model = baseline_model()
# Fit the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
100s - loss: 1.9713 - acc: 0.3007 - val_loss: 1.8439 - val_acc: 0.3543
Epoch 2/10
98s - loss: 1.8009 - acc: 0.3653 - val_loss: 1.7556 - val_acc: 0.3815
Epoch 3/10
91s - loss: 1.7218 - acc: 0.3931 - val_loss: 1.6891 - val_acc: 0.4046
Epoch 4/10
91s - loss: 1.6639 - acc: 0.4138 - val_loss: 1.6320 - val_acc: 0.4263
Epoch 5/10
92s - loss: 1.6177 - acc: 0.4308 - val_loss: 1.6055 - val_acc: 0.4338
Epoch 6/10
91s - loss: 1.5834 - acc: 0.4419 - val_loss: 1.5750 - val_acc: 0.4508
Epoch 7/10
91s - loss: 1.5525 - acc: 0.4542 - val_loss: 1.5488 - val_acc: 0.4580
Epoch 8/10
91s - loss: 1.5276 - acc: 0.4629 - val_loss: 1.5307 - val_acc: 0.4612
Epoch 9/10
92s - loss: 1.5062 - acc: 0.4709 - val_loss: 1.5084 - val_acc: 0.4707
Epoch 10/10
91s - loss: 1.4866 - acc: 0.4771 - val_loss: 1.4926 - val_acc: 0.4792


Baseline Error: 52.08%


In [24]:
num_pixels = 32 * 32 * 3   # 3072
num_classes = 10

from keras.optimizers import Adam

# define baseline model
def baseline_plus_hidden():
    # create model
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
    # Compile model
    adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

# build the model
model = baseline_plus_hidden()
# Fit the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
93s - loss: 2.0939 - acc: 0.2115 - val_loss: 1.9345 - val_acc: 0.2927
Epoch 2/10
89s - loss: 1.9539 - acc: 0.2811 - val_loss: 1.8822 - val_acc: 0.3268
Epoch 3/10
89s - loss: 1.9096 - acc: 0.3003 - val_loss: 1.8281 - val_acc: 0.3425
Epoch 4/10
89s - loss: 1.8751 - acc: 0.3154 - val_loss: 1.8142 - val_acc: 0.3445
Epoch 5/10
89s - loss: 1.8476 - acc: 0.3290 - val_loss: 1.7617 - val_acc: 0.3656
Epoch 6/10
89s - loss: 1.8224 - acc: 0.3366 - val_loss: 1.7270 - val_acc: 0.3800
Epoch 7/10
88s - loss: 1.7978 - acc: 0.3505 - val_loss: 1.7152 - val_acc: 0.3812
Epoch 8/10
89s - loss: 1.7811 - acc: 0.3559 - val_loss: 1.6802 - val_acc: 0.4033
Epoch 9/10
89s - loss: 1.7612 - acc: 0.3641 - val_loss: 1.6617 - val_acc: 0.4070
Epoch 10/10
89s - loss: 1.7494 - acc: 0.3694 - val_loss: 1.6434 - val_acc: 0.4135


Baseline Error: 58.65%


In [25]:
num_pixels = 32 * 32 * 3   # 3072
num_classes = 10

from keras.optimizers import Adam

# define baseline model
def baseline_plus_hidden():
    # create model
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
    # Compile model
    adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

# build the model
model = baseline_plus_hidden()
# Fit the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
97s - loss: 2.2646 - acc: 0.1595 - val_loss: 2.1757 - val_acc: 0.2011
Epoch 2/10
91s - loss: 2.1155 - acc: 0.2015 - val_loss: 2.0545 - val_acc: 0.2521
Epoch 3/10
90s - loss: 2.0453 - acc: 0.2362 - val_loss: 1.9957 - val_acc: 0.2803
Epoch 4/10
90s - loss: 2.0002 - acc: 0.2583 - val_loss: 1.9531 - val_acc: 0.2946
Epoch 5/10
89s - loss: 1.9686 - acc: 0.2740 - val_loss: 1.9255 - val_acc: 0.3062
Epoch 6/10
89s - loss: 1.9516 - acc: 0.2825 - val_loss: 1.9076 - val_acc: 0.3120
Epoch 7/10
89s - loss: 1.9357 - acc: 0.2923 - val_loss: 1.8935 - val_acc: 0.3189
Epoch 8/10
89s - loss: 1.9218 - acc: 0.2991 - val_loss: 1.8787 - val_acc: 0.3208
Epoch 9/10
89s - loss: 1.9071 - acc: 0.3066 - val_loss: 1.8670 - val_acc: 0.3248
Epoch 10/10
90s - loss: 1.8976 - acc: 0.3110 - val_loss: 1.8522 - val_acc: 0.3332


Baseline Error: 66.68%


In [27]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
# from keras import backend as K
# K.set_image_dim_ordering('tf')

In [74]:
# # load data
# (X_train, y_train), (X_test, y_test) = cifar10.load_data()
# # reshape to be [samples][pixels][width][height]
# X_train = X_train.reshape(X_train.shape[0], 3, 32, 32).astype('float32')
# X_test = X_test.reshape(X_test.shape[0], 3, 32, 32).astype('float32')

In [19]:
# # normalize inputs from 0-255 to 0-1
# x_train = x_train / 255
# x_test = x_test / 255
# # one hot encode outputs
# y_train = np_utils.to_categorical(y_train)
# y_test = np_utils.to_categorical(y_test)
# num_classes = y_train.shape[1]
# num_classes

10

In [45]:
def convolution_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(32, 32, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    # Compile model
    adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [49]:
X_train = x_train.reshape(50000, 32, 32, 3)
X_test = x_test.reshape(10000, 32, 32, 3)

In [50]:
# build the model
model = convolution_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

ValueError: Input 0 is incompatible with layer conv2d_9: expected ndim=4, found ndim=3

The following code was adapted from:
https://notebooks.azure.com/anon-yqopra/libraries/pydata/html/workspace.ipynb

In [ ]:
len(x_train), len(y_train), len(x_test), len(y_test)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

def display_images_and_labels(images, labels):
    """Display the first image of each label."""
    unique_labels = set(labels)
    plt.figure(figsize=(15, 15))
    i = 1
    for label in unique_labels:
        # Pick the first image for each label.
        image = images[labels.index(label)]
        plt.subplot(8, 8, i)  # A grid of 8 rows x 8 columns
        plt.axis('off')
        plt.title("Label {0} ({1})".format(label, labels.count(label)))
        i += 1
        _ = plt.imshow(image)
    plt.show()

In [ ]:
display_images_and_labels(x_train, y_train)